In [1]:
RANDOM_STATE = 1234

In [2]:
import numpy as np
import pandas as pd
import torch
from data.featurization.dgl_Graph import DGL_Graph
from model.dgl.GCN import GCN
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [3]:
torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_STATE)

In [4]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
trn = pd.read_csv("../data/processed/trn.csv")
tst_08SC = pd.read_csv("../data/processed/tst.08SC.csv")
tst1_19SC = pd.read_csv("../data/processed/tst.19SC.1.csv")
tst2_19SC = pd.read_csv("../data/processed/tst.19SC.2.csv")

trn_y, trn_X = trn["Solubility"], trn["SMILES"]
tst_08SC_y, tst_08SC_X = tst_08SC["Solubility"], tst_08SC["SMILES"]
tst1_19SC_y, tst1_19SC_X = tst1_19SC["Solubility"], tst1_19SC["SMILES"]
tst2_19SC_y, tst2_19SC_X = tst2_19SC["Solubility"], tst2_19SC["SMILES"]

In [6]:
featurizer = DGL_Graph(
    graph_type="BI_GRAPH",
    featurize_type="Canonical",
    self_loop=True
)
trn_X = featurizer.convert(trn_X)
tst_08SC_X = featurizer.convert(tst_08SC_X)
tst1_19SC_X = featurizer.convert(tst1_19SC_X)
tst2_19SC_X = featurizer.convert(tst2_19SC_X)

In [7]:
tune_space = {
    "hidden_feats": hp.choice("hidden_feats",
                              [[64], [64, 64], [128, 64], [256, 128, 64],
                               [128, 128, 64], [256, 128, 128]]),
    "gnn_norm": hp.choice("gnn_norm", ['right', 'both', 'none']),
    "residual": hp.choice("residual", [True, False]),
    "batchnorm": hp.choice("batchnorm", [True, False]),
    "dropout": hp.uniform("dropout", 0, 1),
    "lr": hp.choice("lr", [0.1, 0.01, 0.001]),
    "weight_decay": hp.uniform("weight_decay", 0, 1),

    "batch_size": 4096,
}

In [8]:
def tune_obj(space):
    kf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
    rmse = []

    for trn_idx, val_idx in kf.split(trn_X):
        tX, vX = trn_X[trn_idx], trn_X[val_idx]
        ty, vy = trn_y[trn_idx], trn_y[val_idx]

        model = GCN(
            n_tasks=1,
            in_feats=featurizer.get_node_feat_size(),
            hidden_feats=space["hidden_feats"],
            gnn_norm=space["gnn_norm"],
            activation=torch.nn.functional.relu,
            residual=space["residual"],
            batchnorm=space["batchnorm"],
            dropout=space["dropout"],
            classifier_hidden_feats=128,
            classifier_dropout=0,
            predictor_hidden_feats=128,
            predictor_dropout=0,
            lr=space["lr"],
            weight_decay=space["weight_decay"],
            batch_size=4096
        )

        scores = model.fit(tX, ty,
                           val_X=vX, val_y=vy,
                           epochs=800, min_epochs=500, early_stop_epochs=10, verbose=False)
        rmse.append(scores["rmse"][-1])

    return {"loss": np.mean(rmse), 'status': STATUS_OK}


In [9]:
trials = Trials()
best_params = fmin(
    fn=tune_obj,
    space=tune_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

[DEVICE] cuda                                          
[DEVICE] cuda                                          
[DEVICE] cuda                                          
[DEVICE] cuda                                                                       
[DEVICE] cuda                                                                       
[DEVICE] cuda                                                                       
[DEVICE] cuda                                                                       
[DEVICE] cuda                                                                       
[DEVICE] cuda                                                                       
[DEVICE] cuda                                                                       
[DEVICE] cuda                                                                       
[DEVICE] cuda                                                                       
[DEVICE] cuda                                                      

In [10]:
best_params

{'batchnorm': 1,
 'dropout': 0.07075484753350235,
 'gnn_norm': 1,
 'hidden_feats': 3,
 'lr': 2,
 'residual': 1,
 'weight_decay': 0.003975036016795143}

In [7]:
best_params = {
    "hidden_feats": [256, 128, 64],
    "gnn_norm": 'both',
    "residual": True,
    "batchnorm": True,
    "dropout": 0.07075484753350235,
    "lr": 0.001,
    "weight_decay": 0.003975036016795143,
    "batch_size": 4096,
}

In [26]:
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

pred_19t1, pred_19t2, pred_08t = [], [], []

for trn_idx, val_idx in kf.split(trn_X):
    tX, vX = trn_X[trn_idx], trn_X[val_idx]
    ty, vy = trn_y[trn_idx], trn_y[val_idx]

    model = GCN(
        n_tasks=1,
        in_feats=featurizer.get_node_feat_size(),
        hidden_feats=best_params["hidden_feats"],
        gnn_norm=best_params["gnn_norm"],
        activation=torch.nn.functional.relu,
        residual=best_params["residual"],
        batchnorm=best_params["batchnorm"],
        dropout=best_params["dropout"],
        classifier_hidden_feats=128,
        classifier_dropout=0,
        predictor_hidden_feats=128,
        predictor_dropout=0,
        lr=best_params["lr"],
        weight_decay=best_params["weight_decay"],
        batch_size=4096
    )

    scores = model.fit(tX, ty, val_X=vX, val_y=vy, epochs=800, min_epochs=400, early_stop_epochs=10)

    pred_19t1.append(model.predict(tst1_19SC_X).cpu())
    pred_19t2.append(model.predict(tst2_19SC_X).cpu())
    pred_08t.append(model.predict(tst_08SC_X).cpu())

[DEVICE] cuda


 54%|█████▎    | 429/800 [00:24<00:21, 17.50it/s, trn_loss:0.016 val_loss:0.155 rmse:0.583 r2:0.913]


[DEVICE] cuda


 59%|█████▉    | 474/800 [00:27<00:18, 17.52it/s, trn_loss:0.016 val_loss:0.166 rmse:0.661 r2:0.884]


[DEVICE] cuda


 55%|█████▍    | 437/800 [00:25<00:20, 17.36it/s, trn_loss:0.018 val_loss:0.184 rmse:0.680 r2:0.867]


[DEVICE] cuda


 56%|█████▋    | 450/800 [00:25<00:20, 17.33it/s, trn_loss:0.015 val_loss:0.176 rmse:0.641 r2:0.904]


[DEVICE] cuda


 58%|█████▊    | 461/800 [00:25<00:19, 17.79it/s, trn_loss:0.015 val_loss:0.185 rmse:0.654 r2:0.907]


In [27]:
pred_19t1 = [torch.mean(pred_i).item() for pred_i in torch.cat(pred_19t1, 1)]
pred_19t2 = [torch.mean(pred_i).item() for pred_i in torch.cat(pred_19t2, 1)]
pred_08t = [torch.mean(pred_i).item() for pred_i in torch.cat(pred_08t, 1)]

In [28]:
performance = {
    "2019SC TST1 (RMSE)": mean_squared_error(tst1_19SC_y, pred_19t1, squared=False),
    "2019SC TST2 (RMSE)": mean_squared_error(tst2_19SC_y, pred_19t2, squared=False),
    "2008SC TST  (RMSE)": mean_squared_error(tst_08SC_y, pred_08t, squared=False),
    "2019SC TST1 (R^2)": r2_score(tst1_19SC_y, pred_19t1),
    "2019SC TST2 (R^2)": r2_score(tst2_19SC_y, pred_19t2),
    "2008SC TST  (R^2)": r2_score(tst_08SC_y, pred_08t),
}
performance

{'2019SC TST1 (RMSE)': 0.7033446436245765,
 '2019SC TST2 (RMSE)': 1.6627868685836469,
 '2008SC TST  (RMSE)': 0.9440606250900933,
 '2019SC TST1 (R^2)': 0.5925717029212769,
 '2019SC TST2 (R^2)': 0.31272448977527845,
 '2008SC TST  (R^2)': 0.5160519748320276}

In [27]:
import matplotlib.pyplot as plt
import seaborn as sns

model_name = "GCN"
min_ax1, max_ax1 = -7, -1
min_ax2, max_ax2 = -10.5, -1
min_ax3, max_ax3 = -8, -1

fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))

sns.lineplot(x=[min_ax1, max_ax1], y=[min_ax1, max_ax1], ax=ax1, color="black")
sns.lineplot(x=[min_ax2, max_ax2], y=[min_ax2, max_ax2], ax=ax2, color="black")
sns.lineplot(x=[min_ax3, max_ax3], y=[min_ax3, max_ax3], ax=ax3, color="black")

sns.regplot(
    data=pd.DataFrame({
        "True log$S$ (mol/L)": tst1_19SC_y.values,
        "Predicted log$S$ (mol/L)": pred_19t1
    }),
    x="True log$S$ (mol/L)",
    y="Predicted log$S$ (mol/L)",
    ax=ax1
)
sns.regplot(
    data=pd.DataFrame({
        "True log$S$ (mol/L)": tst2_19SC_y.values,
        "Predicted log$S$ (mol/L)": pred_19t2
    }),
    x="True log$S$ (mol/L)",
    y="Predicted log$S$ (mol/L)",
    ax=ax2
)
sns.regplot(
    data=pd.DataFrame({
        "True log$S$ (mol/L)": tst_08SC_y.values,
        "Predicted log$S$ (mol/L)": pred_08t
    }),
    x="True log$S$ (mol/L)",
    y="Predicted log$S$ (mol/L)",
    ax=ax3
)

ax1.set_xlim(min_ax1, max_ax1)
ax1.set_ylim(min_ax1, max_ax1)
ax2.set_xlim(min_ax2, max_ax2)
ax2.set_ylim(min_ax2, max_ax2)
ax3.set_xlim(min_ax3, max_ax3)
ax3.set_ylim(min_ax3, max_ax3)

ax1.set_title(f"2019SC Tst1 ({model_name})\n"
              f"RMSE: {performance['2019SC TST1 (RMSE)']:.3f}, "
              f"$R^2$: {performance['2019SC TST1 (R^2)']:.3f}")
ax2.set_title(f"2019SC Tst2 ({model_name})\n"
              f"RMSE: {performance['2019SC TST2 (RMSE)']:.3f}, "
              f"$R^2$: {performance['2019SC TST2 (R^2)']:.3f}")
ax3.set_title(f"2008SC Tst  ({model_name})\n"
              f"RMSE: {performance['2008SC TST  (RMSE)']:.3f}, "
              f"$R^2$: {performance['2008SC TST  (R^2)']:.3f}")

plt.axis("equal")
plt.tight_layout()

ModuleNotFoundError: No module named 'matplotlib'